<link href="https://fonts.googleapis.com/css2?family=Barlow:wght@300&display=swap" rel="stylesheet">
<div style="font-family:'Barlow';font-size:18px;">
<h1>SB - Property Sales</h1> 
<h2>Data Cleaning & Feature Engineering</h2>
</div>

In [1]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import math
import matplotlib.pyplot as plt 

In [3]:
# pd.set_option("display.precision", 4)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.4f' % x)
sns.set_context('talk')
sns.set_style('darkgrid', {'font.sans-serif': ['Iosevka']})

<link href="https://fonts.googleapis.com/css2?family=Barlow:wght@300&display=swap" rel="stylesheet">
<div style="font-family:'Barlow';font-size:18px;">
<h2>Data Summary</h2>
</div>

In [4]:
file = 'datasets/COMP_SALE_SB_EXTRACT_20201019.csv'

In [5]:
def load_dataset(filename, na_val=None, parse_date=False, verbose=False):
    if parse_date:
        to_parse = [1,2]
        df = pd.read_csv(filename, parse_dates=to_parse, na_values=na_val)
    else:
        df = pd.read_csv(filename, na_values=na_val)
    # Features and Target column name 
    features = df.columns
    target = ''

    print("+ Dataset:")
    display(df.head(3))
    
    if verbose:
        print("+ Shape:")
        display(df.shape) 
        print(f"+ Target: {target}")
        print(f"+ Overall Variables Info: ")
        display(pd.DataFrame(df.info(memory_usage='deep')))
    
    return features, target, df

features, target, dataset = load_dataset(file, na_val=None, parse_date=False, verbose=False)

+ Dataset:


,ID,CREATED_AT,SOLD_DATE,SOLD_DURATION,ADDRESS,LATITUDE,LONGITUDE,SOLD_PRICE,UNIT_NUMBER,PROPERTY_FLOOR,BUILDING_SIZE,BUILDING_SURFACE,LOT_SURFACE,LAND_REGISTER,BUILDING_YEAR,LOT_EVALUATION,BUILDING_EVALUATION,YEAR_EVALUATION,RESIDENTIAL_REVENUE,COMMERCIAL_REVENUE,PARKING_REVENUE,OTHERS_REVENUE,MUNICIPAL_TAXE,SCHOOL_TAXE,EXPENSES_ELECTRICITY,EXPENSES_OIL,EXPENSES_GAZ,EXPENSES_ASSURANCE,EXPENSES_SNOW,EXPENSES_MAINTENANCE,FONDATION_TYPE,HEATING_ENERGY,NUMBER_1_H,NUMBER_2_H,NUMBER_3_H,NUMBER_4_H,NUMBER_5_H,NUMBER_6_H,NUMBER_7_H,NUMBER_8_H,NUMBER_9_H,AGENCY_NAME,AGENCY_ADDRESS,CT_FULL_NAME,CT_EMAIL,DISTANCE
0,101,2020-05-29 08:24:07,2018-12-31,381,431-433 12e Avenue N. Sherbrooke (Fleurimont) ...,45.4506,-71.8607,125000,2,2,29.9 X 35.2 ft / 9.07 X 10.72 m,"1,047.6 sqft / 97.33 sqm","6,124.66 sqft / 569 sqm",1 329 778 Cadastre du Quebec,1952,71100,98400,2014,15600,0,0,0,2386,267,0,0,0,0,0,0,Concrete slab on ground,"Electricity, Other",0,0,0,0,2,0,0,0,0,RE/MAX D'ABORD INC.,157 boul. Jacques-Cartier S. Sherbrooke J1J 2Z4,Katy Rheaume,katyrheaume@remax-quebec.com,9.0569
1,102,2020-05-29 08:24:13,2019-01-15,323,118 5e Avenue Sherbrooke (Fleurimont) J1G 2L5,45.4037,-71.8777,369000,2,0,37 X 44 ft irr / 11.28 X 13.41 m irr,"1,630 sqft / 151.43 sqm","6,476 sqft / 601.64 sqm",1132557,1949,75100,206200,2018,1,0,0,0,3476,803,4694,0,0,0,0,0,Poured concrete,"Dual energy, Electricity, Heating oil",0,0,0,0,0,0,1,0,0,CENTURY 21 ACTIONWEB INC.,25 rue Bocage Suite 102 Sherbrooke J1L 2J4,Patrick Fares,patrick.fares@century21.ca,4.8913
2,103,2020-05-29 08:24:19,2019-01-23,86,502 Rue King E. Sherbrooke (Fleurimont) J1G 1B5,45.4079,-71.8777,340000,6,3,30.1 X 41.1 ft / 9.16 X 12.52 m,"1,234.4 sqft / 114.68 sqm","4,500.39 sqft / 418.1 sqm",1330580,1972,104500,225300,2018,28452,0,0,0,5164,562,371,0,0,1426,500,0,Poured concrete,Electricity,0,0,6,0,0,0,0,0,0,Via Capitale Estrie,30 Rioux Sherbrooke J1J 2W7,Dominic Glaude,dominic.glaude@gmail.com,5.0672


In [6]:
def display_missing(dataset, verbose=False):
    features = dataset.columns
    if verbose:
        print("+ Missing data: ")
    missing_count = dataset[features].isnull().sum()
    missing_percent = (dataset[features].isnull().sum() / len(dataset) * 100).round(4)
    missing = pd.DataFrame({'feature': features, 'count': missing_count, 'percent': missing_percent,})
    missing.sort_values('count', ascending=False, inplace=True)
    missing.reset_index(drop=True, inplace=True)
    return missing

display_missing(dataset, True)

+ Missing data: 


,feature,count,percent
0,BUILDING_SURFACE,397,26.4667
1,FONDATION_TYPE,209,13.9333
2,HEATING_ENERGY,172,11.4667
3,BUILDING_SIZE,100,6.6667
4,LOT_SURFACE,38,2.5333
5,NUMBER_5_H,0,0.0000
6,EXPENSES_ASSURANCE,0,0.0000
7,EXPENSES_SNOW,0,0.0000
8,EXPENSES_MAINTENANCE,0,0.0000
9,NUMBER_1_H,0,0.0000


## 1. Feature Extraction: extracting KPIs

In [7]:
# KPI Extractor functions
def get_rba(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    return row["RESIDENTIAL_REVENUE"] + row["COMMERCIAL_REVENUE"] + row["PARKING_REVENUE"] + row["OTHERS_REVENUE"]

def get_round_up(row):
    if np.isnan(row["RBA"]):
        return np.nan
    decimals = -3
    multiplier = 10 ** decimals
    return math.ceil(row["RBA"] * multiplier) / multiplier

def get_revenue_gross_round(row):
    if np.isnan(row["RBA"]):
        return np.nan
    return round(row["RBA"], -3)

def get_norm_expense_vac(row):
    if np.isnan(row["RBA"]):
        return np.nan
    return round(row["RBA"] * 0.03, 2)

def get_norm_expense_ins(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if np.isnan(row["EXPENSES_ASSURANCE"]) or row["EXPENSES_ASSURANCE"] < 100:
        if row["UNIT_NUMBER"] > 5:
            return row["UNIT_NUMBER"] * 500
        else:
            return row["UNIT_NUMBER"] * 650
    else:
        return row["EXPENSES_ASSURANCE"]

def get_norm_expense_maint(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if row["UNIT_NUMBER"] >= 5:
        return row["UNIT_NUMBER"] * 500
    else:
        return np.nan

def get_norm_expense_conc(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if 4 <= row["UNIT_NUMBER"] <= 6:
        row = row["UNIT_NUMBER"] * 125
    elif 7 <= row["UNIT_NUMBER"] <= 11:
        row = row["UNIT_NUMBER"] * 170
    else:
        row = row["UNIT_NUMBER"] * 300
    return row

def get_norm_expense_mngt(row):
    if np.isnan(row["RBA"]) or np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if 5 <= row["UNIT_NUMBER"] <= 6:
        row = row["RBA"] * .03
    elif 7 <= row["UNIT_NUMBER"] <= 11:
        row = row["RBA"] * .04
    else:
        row = row["RBA"] * .05
    return row

def get_norm_expense_elec(row):
    if np.isnan(row["EXPENSES_ELECTRICITY"]):
        return np.nan
    else:
        return row["EXPENSES_ELECTRICITY"]

def get_norm_expense_heat(row):
    if np.isnan(row["EXPENSES_OIL"]) or np.isnan(row["EXPENSES_GAZ"]):
        if not np.isnan(row["EXPENSES_OIL"]):
            return row["EXPENSES_OIL"]
        if not np.isnan(row["EXPENSES_GAZ"]):
            return row["EXPENSES_GAZ"]
        else:
            return np.nan
    return row["EXPENSES_OIL"] + row["EXPENSES_GAZ"]

def get_total_norm_expenses(row):
    if np.isnan(row["UNIT_NUMBER"]) or np.isnan(row["RBA"]):
        return np.nan
    total_exp = 0
    if row["UNIT_NUMBER"] >= 5:
        total_exp = \
            row["MUNICIPAL_TAXE"] + row["SCHOOL_TAXE"] + \
            row["NORM_EXP_MAINTENANCE"] + row["NORM_EXP_CONCIERGE"] + \
            row["NORM_EXP_MANAGEMENT"] + row["NORM_EXP_INSURANCE"] + \
            row["NORM_EXP_ELECTRICITY"] + row["NORM_EXP_HEAT"] + \
            row["NORM_EXP_VACANCY"]

    if 2 <= row["UNIT_NUMBER"] <= 4 and row["RBA"] > 5000:
        total_exp = \
            row["MUNICIPAL_TAXE"] + row["SCHOOL_TAXE"] + \
            row["EXPENSES_ASSURANCE"] + row["NORM_EXP_INSURANCE"] + \
            row["NORM_EXP_ELECTRICITY"] + row["NORM_EXP_HEAT"] + \
            row["NORM_EXP_VACANCY"] + row["EXPENSES_SNOW"]
    return total_exp

def get_total_norm_expenses_rounded(row):
    if np.isnan(row["NORM_TOTAL_EXPENSES"]):
        return np.nan
    return abs(round(row["NORM_TOTAL_EXPENSES"], -2))

def get_net_norm_income(row):
    if np.isnan(row["RBA"]) or np.isnan(row["NORM_TOTAL_EXPENSES"]):
        return np.nan
    return round(row["RBA"] - row["NORM_TOTAL_EXPENSES"], 2)

def get_cap_rate(row):
    if np.isnan(row["NET_NORM_INCOME"]) or np.isnan(row["SOLD_PRICE"]):
        return np.nan
    return round((row["NET_NORM_INCOME"] / row["SOLD_PRICE"] * 100), 2)

def get_gross_revenue_mult(row):
    if np.isnan(row["SOLD_PRICE"]) or np.isnan(row["RBA"]) or row["RBA"] == 0:
        return np.nan
    return round(row["SOLD_PRICE"] / row["RBA"], 2)

def get_unit_price(row):
    if np.isnan(row["SOLD_PRICE"]) or np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if row["UNIT_NUMBER"] == 0:
        return row["UNIT_NUMBER"]
    return round(row["SOLD_PRICE"] / row["UNIT_NUMBER"], 2)

def get_net_revenue_mult(row):
    if np.isnan(row["SOLD_PRICE"]) or np.isnan(row["NET_NORM_INCOME"]):
        return np.nan
    if row["NET_NORM_INCOME"] == 0:
        return row["SOLD_PRICE"]
    return round(row["SOLD_PRICE"] / row["NET_NORM_INCOME"], 2)

In [8]:
df = dataset.copy()
df["RBA"] = df.apply(get_rba, axis=1)
df["REVENUE_GROSS_ROUND"] = df.apply(get_revenue_gross_round, axis=1)
df["NORM_EXP_HEAT"] = df.apply(get_norm_expense_heat, axis=1)
df["NORM_EXP_VACANCY"] = df.apply(get_norm_expense_vac, axis=1)
df["NORM_EXP_INSURANCE"] = df.apply(get_norm_expense_ins, axis=1)
df["NORM_EXP_MAINTENANCE"] = df.apply(get_norm_expense_maint, axis=1)
df["NORM_EXP_CONCIERGE"] = df.apply(get_norm_expense_conc, axis=1)
df["NORM_EXP_MANAGEMENT"] = df.apply(get_norm_expense_mngt, axis=1)
df["NORM_EXP_ELECTRICITY"] = df.apply(get_norm_expense_elec, axis=1)
df["NORM_TOTAL_EXPENSES"] = df.apply(get_total_norm_expenses, axis=1)
df["TOTAL_NORM_EXP_ROUNDED"] = df.apply(get_total_norm_expenses_rounded, axis=1)
df["NET_NORM_INCOME"] = df.apply(get_net_norm_income, axis=1)
df["CAP_RATE"] = df.apply(get_cap_rate, axis=1)
df["GROSS_REVENUE_MULTIPLICATOR"] = df.apply(get_gross_revenue_mult, axis=1)
df["UNIT_PRICE"] = df.apply(get_unit_price, axis=1)
df["NET_REVENUE_MULTIPLICATOR"] = df.apply(get_net_revenue_mult, axis=1)

df.to_csv(r'datasets/COMP_SALE_SB_EXTRACT_KPI_20201020.csv', index=False)

In [9]:
# Cross check the data 
file = 'datasets/COMP_SALE_SB_EXTRACT_KPI_20201020.csv' 
features, target, dataset = load_dataset(file, na_val=None, parse_date=False,)


+ Dataset:


,ID,CREATED_AT,SOLD_DATE,SOLD_DURATION,ADDRESS,LATITUDE,LONGITUDE,SOLD_PRICE,UNIT_NUMBER,PROPERTY_FLOOR,BUILDING_SIZE,BUILDING_SURFACE,LOT_SURFACE,LAND_REGISTER,BUILDING_YEAR,LOT_EVALUATION,BUILDING_EVALUATION,YEAR_EVALUATION,RESIDENTIAL_REVENUE,COMMERCIAL_REVENUE,PARKING_REVENUE,OTHERS_REVENUE,MUNICIPAL_TAXE,SCHOOL_TAXE,EXPENSES_ELECTRICITY,EXPENSES_OIL,EXPENSES_GAZ,EXPENSES_ASSURANCE,EXPENSES_SNOW,EXPENSES_MAINTENANCE,FONDATION_TYPE,HEATING_ENERGY,NUMBER_1_H,NUMBER_2_H,NUMBER_3_H,NUMBER_4_H,NUMBER_5_H,NUMBER_6_H,NUMBER_7_H,NUMBER_8_H,NUMBER_9_H,AGENCY_NAME,AGENCY_ADDRESS,CT_FULL_NAME,CT_EMAIL,DISTANCE,RBA,REVENUE_GROSS_ROUND,NORM_EXP_HEAT,NORM_EXP_VACANCY,NORM_EXP_INSURANCE,NORM_EXP_MAINTENANCE,NORM_EXP_CONCIERGE,NORM_EXP_MANAGEMENT,NORM_EXP_ELECTRICITY,NORM_TOTAL_EXPENSES,TOTAL_NORM_EXP_ROUNDED,NET_NORM_INCOME,CAP_RATE,GROSS_REVENUE_MULTIPLICATOR,UNIT_PRICE,NET_REVENUE_MULTIPLICATOR
0,101,2020-05-29 08:24:07,2018-12-31,381,431-433 12e Avenue N. Sherbrooke (Fleurimont) ...,45.4506,-71.8607,125000,2,2,29.9 X 35.2 ft / 9.07 X 10.72 m,"1,047.6 sqft / 97.33 sqm","6,124.66 sqft / 569 sqm",1 329 778 Cadastre du Quebec,1952,71100,98400,2014,15600,0,0,0,2386,267,0,0,0,0,0,0,Concrete slab on ground,"Electricity, Other",0,0,0,0,2,0,0,0,0,RE/MAX D'ABORD INC.,157 boul. Jacques-Cartier S. Sherbrooke J1J 2Z4,Katy Rheaume,katyrheaume@remax-quebec.com,9.0569,15600,16000,0,468.0000,1300,nan,600,780.0000,0,4421.0000,4400.0000,11179.0000,8.9400,8.0100,62500.0000,11.1800
1,102,2020-05-29 08:24:13,2019-01-15,323,118 5e Avenue Sherbrooke (Fleurimont) J1G 2L5,45.4037,-71.8777,369000,2,0,37 X 44 ft irr / 11.28 X 13.41 m irr,"1,630 sqft / 151.43 sqm","6,476 sqft / 601.64 sqm",1132557,1949,75100,206200,2018,1,0,0,0,3476,803,4694,0,0,0,0,0,Poured concrete,"Dual energy, Electricity, Heating oil",0,0,0,0,0,0,1,0,0,CENTURY 21 ACTIONWEB INC.,25 rue Bocage Suite 102 Sherbrooke J1L 2J4,Patrick Fares,patrick.fares@century21.ca,4.8913,1,0,0,0.0300,1300,nan,600,0.0500,4694,0.0000,0.0000,1.0000,0.0000,369000.0000,184500.0000,369000.0000
2,103,2020-05-29 08:24:19,2019-01-23,86,502 Rue King E. Sherbrooke (Fleurimont) J1G 1B5,45.4079,-71.8777,340000,6,3,30.1 X 41.1 ft / 9.16 X 12.52 m,"1,234.4 sqft / 114.68 sqm","4,500.39 sqft / 418.1 sqm",1330580,1972,104500,225300,2018,28452,0,0,0,5164,562,371,0,0,1426,500,0,Poured concrete,Electricity,0,0,6,0,0,0,0,0,0,Via Capitale Estrie,30 Rioux Sherbrooke J1J 2W7,Dominic Glaude,dominic.glaude@gmail.com,5.0672,28452,28000,0,853.5600,1426,3000.0000,750,853.5600,371,12980.1200,13000.0000,15471.8800,4.5500,11.9500,56666.6700,21.9800


## 2. Feature Extraction: cleaning and creating new features 

In [10]:
def get_bldg_surface(row):
    """ This will extract the building surface in Square Feet meters 
    """
    if pd.isna(row["BUILDING_SURFACE"]):
        return np.nan
    return float(row["BUILDING_SURFACE"].split()[0].replace(',', ''))


def get_lot_surface(row):
    """ This will extract the lot surface in Square Feet meters 
    """
    if pd.isna(row["LOT_SURFACE"]):
        return np.nan
    return float(row["LOT_SURFACE"].split()[0].replace(',', ''))


def get_fondation_quality(row):
    if pd.isna(row["FONDATION_TYPE"]):
        return np.nan
    elif "Other" in row["FONDATION_TYPE"].strip():
        return 1
    elif ("Stone" or \
            "Stone, Other" or \
            "Concrete blocks, Stone, Other" or \
            "Poured concrete, Stone, Other" or \
            "Concrete blocks, Stone") in row["FONDATION_TYPE"].strip():
        return 3
    elif ("Concrete slab on ground, Poured concrete, Stone" or \
            "Concrete slab on ground, Stone" or \
            "Concrete") in row["FONDATION_TYPE"].strip():
        return 4
    elif ("Poured concrete, Stone" or \
            "Concrete blocks, Poured concrete, Stone") in row["FONDATION_TYPE"].strip():
        return 5
    elif ("Concrete slab on ground" or \
            "Concrete blocks" or \
            "Concrete slab on ground, Poured concrete" or \
            "Concrete blocks, Poured concrete, Other" or \
            "Concrete blocks, Concrete slab on ground" or \
            "Concrete slab on ground, Other" or \
            "Concrete blocks, Other") in \
          row["FONDATION_TYPE"].strip():
        return 6
    elif ("Poured concrete, Other" or "Concrete") in row["FONDATION_TYPE"].strip():
        return 7
    elif ("Poured concrete" or "Concrete blocks, Poured concrete") in row["FONDATION_TYPE"].strip():
        return 8
    else:
        return np.nan


def get_age_evaluation(row):
    """ Age evaluation is the difference between the year of the evaluation and the year when the property is build 
    """
    if np.isnan(row["YEAR_EVALUATION"]) or row["YEAR_EVALUATION"] == 0 or np.isnan(row["BUILDING_YEAR"]) or row["BUILDING_YEAR"] == 0:
        return np.nan
    
    return row["YEAR_EVALUATION"] - row["BUILDING_YEAR"]


def get_norm_sold_duration(row):
    """ Sold duration is converted and ceiled into months: 1 month -> 30 days 
    """
    if pd.isna(row["SOLD_DURATION"]):
        return np.nan
    return np.ceil(np.ceil(row["SOLD_DURATION"] / 30) / 3)

def get_norm_diff_price_eval_sold(row): 
    """ This will return the normalized difference between the sold price and the evaluation price
    """
    if np.isnan(row["SOLD_PRICE"]) or np.isnan(row["LOT_EVALUATION"]) or np.isnan(row["BUILDING_EVALUATION"]):
        return np.nan
    return (row["SOLD_PRICE"] - (row["LOT_EVALUATION"] + row["BUILDING_EVALUATION"])) / row["SOLD_PRICE"]
    
    

In [11]:
# extract sizes  
df = dataset.copy()
df["BUILDING_SURFACE_SQFT"] = df.apply(get_bldg_surface, axis=1)
df["LOT_SURFACE_SQFT"] = df.apply(get_lot_surface, axis=1)
df["FONDATION_QUAL"] = df.apply(get_fondation_quality, axis=1)
df["AGE_EVALUATION"] = df.apply(get_age_evaluation, axis=1)
df["NORM_SOLD_DURATION_TRIM"] = df.apply(get_norm_sold_duration, axis=1)
df["NORM_DIFF_PRICE_EVAL_SOLD"] = df.apply(get_norm_diff_price_eval_sold, axis=1)

## 3. Removing redundant and irrelevant features 

In [12]:
# Remove redundant/unecessary features 
cols_drop = ["ID", "CREATED_AT", "SOLD_DATE", "RBA", "SOLD_DURATION", "YEAR_EVALUATION", "BUILDING_YEAR",
            "BUILDING_SIZE", "BUILDING_SURFACE", "LOT_SURFACE", "LAND_REGISTER", 
            "FONDATION_TYPE", "AGENCY_ADDRESS", "CT_FULL_NAME", "CT_EMAIL", 
            "EXPENSES_MAINTENANCE", "EXPENSES_ELECTRICITY", "EXPENSES_OIL", "EXPENSES_GAZ",
            "EXPENSES_ASSURANCE",  "NORM_EXP_MAINTENANCE", "NORM_TOTAL_EXPENSES"]

# Save dataset having "Salary Estimate"
df.drop(cols_drop, inplace=True, axis=1)
# Save 
df.to_csv('datasets/COMP_SALE_SB_CLEAN_20201114.csv', index=False)

In [13]:
# Cross check the data 
file = 'datasets/COMP_SALE_SB_CLEAN_20201114.csv' 
features, target, dataset = load_dataset(file, na_val=None, parse_date=False)

+ Dataset:


,ADDRESS,LATITUDE,LONGITUDE,SOLD_PRICE,UNIT_NUMBER,PROPERTY_FLOOR,LOT_EVALUATION,BUILDING_EVALUATION,RESIDENTIAL_REVENUE,COMMERCIAL_REVENUE,PARKING_REVENUE,OTHERS_REVENUE,MUNICIPAL_TAXE,SCHOOL_TAXE,EXPENSES_SNOW,HEATING_ENERGY,NUMBER_1_H,NUMBER_2_H,NUMBER_3_H,NUMBER_4_H,NUMBER_5_H,NUMBER_6_H,NUMBER_7_H,NUMBER_8_H,NUMBER_9_H,AGENCY_NAME,DISTANCE,REVENUE_GROSS_ROUND,NORM_EXP_HEAT,NORM_EXP_VACANCY,NORM_EXP_INSURANCE,NORM_EXP_CONCIERGE,NORM_EXP_MANAGEMENT,NORM_EXP_ELECTRICITY,TOTAL_NORM_EXP_ROUNDED,NET_NORM_INCOME,CAP_RATE,GROSS_REVENUE_MULTIPLICATOR,UNIT_PRICE,NET_REVENUE_MULTIPLICATOR,BUILDING_SURFACE_SQFT,LOT_SURFACE_SQFT,FONDATION_QUAL,AGE_EVALUATION,NORM_SOLD_DURATION_TRIM,NORM_DIFF_PRICE_EVAL_SOLD
0,431-433 12e Avenue N. Sherbrooke (Fleurimont) ...,45.4506,-71.8607,125000,2,2,71100,98400,15600,0,0,0,2386,267,0,"Electricity, Other",0,0,0,0,2,0,0,0,0,RE/MAX D'ABORD INC.,9.0569,16000,0,468.0000,1300,600,780.0000,0,4400.0000,11179.0000,8.9400,8.0100,62500.0000,11.1800,1047.6000,6124.6600,6.0000,62.0000,5.0000,-0.3560
1,118 5e Avenue Sherbrooke (Fleurimont) J1G 2L5,45.4037,-71.8777,369000,2,0,75100,206200,1,0,0,0,3476,803,0,"Dual energy, Electricity, Heating oil",0,0,0,0,0,0,1,0,0,CENTURY 21 ACTIONWEB INC.,4.8913,0,0,0.0300,1300,600,0.0500,4694,0.0000,1.0000,0.0000,369000.0000,184500.0000,369000.0000,1630.0000,6476.0000,8.0000,69.0000,4.0000,0.2377
2,502 Rue King E. Sherbrooke (Fleurimont) J1G 1B5,45.4079,-71.8777,340000,6,3,104500,225300,28452,0,0,0,5164,562,500,Electricity,0,0,6,0,0,0,0,0,0,Via Capitale Estrie,5.0672,28000,0,853.5600,1426,750,853.5600,371,13000.0000,15471.8800,4.5500,11.9500,56666.6700,21.9800,1234.4000,4500.3900,8.0000,46.0000,1.0000,0.0300


In [14]:
display(dataset.describe())

,LATITUDE,LONGITUDE,SOLD_PRICE,UNIT_NUMBER,PROPERTY_FLOOR,LOT_EVALUATION,BUILDING_EVALUATION,RESIDENTIAL_REVENUE,COMMERCIAL_REVENUE,PARKING_REVENUE,OTHERS_REVENUE,MUNICIPAL_TAXE,SCHOOL_TAXE,EXPENSES_SNOW,NUMBER_1_H,NUMBER_2_H,NUMBER_3_H,NUMBER_4_H,NUMBER_5_H,NUMBER_6_H,NUMBER_7_H,NUMBER_8_H,NUMBER_9_H,DISTANCE,REVENUE_GROSS_ROUND,NORM_EXP_HEAT,NORM_EXP_VACANCY,NORM_EXP_INSURANCE,NORM_EXP_CONCIERGE,NORM_EXP_MANAGEMENT,NORM_EXP_ELECTRICITY,TOTAL_NORM_EXP_ROUNDED,NET_NORM_INCOME,CAP_RATE,GROSS_REVENUE_MULTIPLICATOR,UNIT_PRICE,NET_REVENUE_MULTIPLICATOR,BUILDING_SURFACE_SQFT,LOT_SURFACE_SQFT,FONDATION_QUAL,AGE_EVALUATION,NORM_SOLD_DURATION_TRIM,NORM_DIFF_PRICE_EVAL_SOLD
count,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1500.0000,1491.0000,1500.0000,1500.0000,1103.0000,1462.0000,1273.0000,1254.0000,1500.0000,1500.0000
mean,45.3828,-71.9268,337628.4307,4.3840,1.1973,67378.9193,259108.0693,31658.7327,802.0620,31.0087,31.0087,4706.0427,726.1873,199.5533,0.2520,0.1720,0.7653,1.5040,0.9267,0.1873,0.1047,0.0753,0.0440,8.0391,32519.3333,632.6407,975.6844,1932.4473,1029.0667,1499.1584,1314.9913,12899.4000,19624.0694,5.6078,2003.2398,88876.5033,3023.5290,1664.5695,17868.3995,6.6614,56.1651,2.1240,-0.0471
std,0.0640,0.1065,378039.1618,6.3457,5.6544,53601.7572,312619.8193,64287.2655,4833.4477,369.0235,369.0235,5066.7301,799.1192,374.3123,3.3280,0.9318,1.7612,2.7644,2.0169,0.4876,0.3509,0.3105,0.2178,7.4941,64488.2169,2611.4281,1934.6171,2161.3568,1892.0012,3207.4084,2648.8205,20324.3711,46689.7092,4.3336,21331.7689,38635.8260,26279.3054,1753.5503,276221.0681,2.1822,30.0311,1.8648,0.3561
min,45.1321,-72.3050,15000.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0580,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-37650.9600,-6.9800,0.6800,0.0000,-3061.8000,1.0000,1.1000,1.0000,0.0000,0.0000,-5.2333
25%,45.3740,-71.9526,180375.0000,2.0000,0.0000,39500.0000,138850.0000,14805.0000,0.0000,0.0000,0.0000,2656.0000,377.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,3.3066,15000.0000,0.0000,451.3500,1300.0000,600.0000,750.0000,0.0000,5600.0000,8990.8000,4.5500,9.4900,64000.0000,15.5525,1033.4900,5000.0000,6.0000,33.0000,1.0000,-0.0998
50%,45.3931,-71.8971,240000.0000,3.0000,0.0000,58100.0000,182450.0000,20430.0000,0.0000,0.0000,0.0000,3368.0000,541.5000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,4.8559,21000.0000,0.0000,631.8000,1300.0000,600.0000,1001.7000,0.0000,7900.0000,13251.4600,5.3000,11.4400,82500.0000,18.7150,1267.0000,7500.0000,8.0000,58.0000,1.0000,0.0147
75%,45.4081,-71.8780,365000.0000,4.0000,2.0000,80425.0000,274950.0000,33072.0000,0.0000,0.0000,0.0000,4993.2500,792.2500,400.0000,0.0000,0.0000,1.0000,2.0000,1.0000,0.0000,0.0000,0.0000,0.0000,9.0569,34000.0000,0.0000,1009.5300,1963.2500,900.0000,1356.1500,1852.0000,13000.0000,20468.0900,6.3325,13.5150,109444.5000,21.7700,1836.0000,11009.4125,8.0000,73.0000,3.0000,0.0997
max,45.6431,-71.5645,6000000.0000,99.0000,215.0000,878200.0000,5460200.0000,1462032.0000,64368.0000,10200.0000,10200.0000,82492.0000,11885.0000,3500.0000,89.0000,22.0000,18.0000,38.0000,42.0000,4.0000,4.0000,4.0000,2.0000,29.9918,1462000.0000,55750.0000,43860.9600,44000.0000,29700.0000,73101.6000,35525.0000,325100.0000,1184812.4400,87.3800,437500.0000,437500.0000,437500.0000,26464.1400,10560467.9600,8.0000,147.0000,21.0000,1.0000
